In [4]:
import pandas as pd
import numpy as np
import os



import zipfile

home = os.getenv('HOME') +'/aiffel/datathon_apt/data/'

def unzip_file(zip_file_path, extract_to_path):
    """
    zip_file_path: 압축 파일의 경로
    extract_to_path: 압축을 푸는 경로
    """
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)

# 사용 예시
zip_file_path = home + 'train.csv.zip' # 압축 파일 경로
extract_to_path = home  # 압축을 푸는 경로

unzip_file(zip_file_path, extract_to_path)



In [6]:



home = os.getenv('HOME')+'/aiffel/datathon_apt/data/'


data_df = pd.read_csv(home+'train.csv')
data_df

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.8200,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.1700,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.7400,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.3900,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.4300,2004,200801,21~31,3,120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216548,1234823,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,59.8245,2012,201711,11~20,3,22000
1216549,1234824,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,7,30750
1216550,1234825,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,9,29500
1216551,1234826,9586,부산광역시,정관읍 용수리,1364,정관신도시롯데캐슬,정관읍 용수리 1364 정관신도시롯데캐슬,101.7400,2008,201711,21~30,7,26700


In [61]:
unique_values = data_df[['city', 'dong']].drop_duplicates()


In [62]:
unique_values

,city,dong
0,서울특별시,신교동
1,서울특별시,필운동
3,서울특별시,내수동
5,서울특별시,인의동
10,서울특별시,이화동
...,...,...
1092821,서울특별시,교남동
1093070,서울특별시,효제동
1184449,부산광역시,충무동1가
1209136,부산광역시,대저1동


In [63]:
# unique_values['dong'].str[:3].tail(10)


In [64]:
unique_values['addr_kr'] = unique_values['city'] + ' ' + unique_values['dong'].apply(lambda x: x.strip().split()[-1])
unique_values

,city,dong,addr_kr
0,서울특별시,신교동,서울특별시 신교동
1,서울특별시,필운동,서울특별시 필운동
3,서울특별시,내수동,서울특별시 내수동
5,서울특별시,인의동,서울특별시 인의동
10,서울특별시,이화동,서울특별시 이화동
...,...,...,...
1092821,서울특별시,교남동,서울특별시 교남동
1093070,서울특별시,효제동,서울특별시 효제동
1184449,부산광역시,충무동1가,부산광역시 충무동1가
1209136,부산광역시,대저1동,부산광역시 대저1동


In [65]:
unique_values['addr_kr'].tail(10)

1074165      부산광역시 모전리
1075539       부산광역시 내리
1083123      부산광역시 고촌리
1090197      부산광역시 명례리
1091655      부산광역시 좌천리
1092821      서울특별시 교남동
1093070      서울특별시 효제동
1184449    부산광역시 충무동1가
1209136     부산광역시 대저1동
1209616      부산광역시 녹산동
Name: addr_kr, dtype: object

In [66]:
from tqdm import tqdm
from geopy.geocoders import Nominatim

# Geopy를 사용하여 위도와 경도를 조회하는 함수
def get_lat_lon(address):
    geolocator = Nominatim(user_agent="chiricuto")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# 'addr_kr' 열의 각 주소에 대해 위도와 경도 조회
# tqdm을 사용하여 진행 상황을 표시
tqdm.pandas()
unique_values[['latitude', 'longitude']] = unique_values['addr_kr'].progress_apply(get_lat_lon).apply(pd.Series)

print(unique_values.head())


100%|██████████| 477/477 [04:00<00:00,  1.99it/s]

     city dong    addr_kr   latitude   longitude
0   서울특별시  신교동  서울특별시 신교동  37.584160  126.967230
1   서울특별시  필운동  서울특별시 필운동  37.577363  126.969097
3   서울특별시  내수동  서울특별시 내수동  37.572940  126.971800
5   서울특별시  인의동  서울특별시 인의동  37.572360  126.997100
10  서울특별시  이화동  서울특별시 이화동  37.580476  127.002547


In [67]:
unique_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477 entries, 0 to 1209616
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city       477 non-null    object 
 1   dong       477 non-null    object 
 2   addr_kr    477 non-null    object 
 3   latitude   473 non-null    float64
 4   longitude  473 non-null    float64
dtypes: float64(2), object(3)
memory usage: 22.4+ KB


In [48]:
unique_values.tail(20)

,city,dong,addr_kr,latitude,longitude
1073133,부산광역시,기장읍 서부리,부산광역시 서부리,35.247400,129.197780
1073140,부산광역시,기장읍 대라리,부산광역시 대라리,35.236560,129.208550
1073163,부산광역시,기장읍 청강리,부산광역시 청강리,35.222880,129.210320
1073177,부산광역시,정관읍 달산리,부산광역시 달산리,35.322250,129.199440
1073247,부산광역시,일광면 삼성리,부산광역시 삼성리,35.266400,129.231303
1073377,부산광역시,일광면 이천리,부산광역시 이천리,35.269231,129.238642
1073579,부산광역시,기장읍 대변리,부산광역시 대변리,35.225990,129.229260
1073974,부산광역시,정관읍 방곡리,부산광역시 방곡리,35.328530,129.188550
1074051,부산광역시,정관읍 용수리,부산광역시 용수리,35.336620,129.173120
1074100,부산광역시,정관읍 매학리,부산광역시 매학리,35.316090,129.168060


In [59]:
unique_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477 entries, 0 to 1209616
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city       477 non-null    object 
 1   dong       477 non-null    object 
 2   addr_kr    477 non-null    object 
 3   latitude   457 non-null    float64
 4   longitude  457 non-null    float64
dtypes: float64(2), object(3)
memory usage: 22.4+ KB


In [54]:
unique_values.isnull()

,city,dong,addr_kr,latitude,longitude
0,False,False,False,False,False
1,False,False,False,False,False
3,False,False,False,False,False
5,False,False,False,False,False
10,False,False,False,False,False
...,...,...,...,...,...
1092821,False,False,False,False,False
1093070,False,False,False,False,False
1184449,False,False,False,False,False
1209136,False,False,False,False,False


In [68]:
unique_values.to_csv('train_la&lo.csv')

In [69]:
unique_values[unique_values[['latitude', 'longitude']].isnull().any(axis=1)] #4건 결측값 확인

,city,dong,addr_kr,latitude,longitude
418969,서울특별시,영등포동7가,서울특별시 영등포동7가,NaN,NaN
652958,부산광역시,서대신동1가,부산광역시 서대신동1가,NaN,NaN
653167,부산광역시,토성동3가,부산광역시 토성동3가,NaN,NaN
667706,부산광역시,신선동3가,부산광역시 신선동3가,NaN,NaN


In [70]:
merged_df = pd.merge(data_df, unique_values[['city', 'dong', 'latitude', 'longitude']], on=['city', 'dong'], how='left')
merged_df

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,latitude,longitude
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.8200,2002,200801,21~31,2,37500,37.584160,126.967230
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.1700,1973,200801,1~10,6,20000,37.577363,126.969097
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.7400,2007,200801,1~10,6,38500,37.577363,126.969097
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.3900,2003,200801,11~20,15,118000,37.572940,126.971800
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.4300,2004,200801,21~31,3,120000,37.572940,126.971800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216548,1234823,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,59.8245,2012,201711,11~20,3,22000,35.336620,129.173120
1216549,1234824,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,7,30750,35.336620,129.173120
1216550,1234825,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,9,29500,35.336620,129.173120
1216551,1234826,9586,부산광역시,정관읍 용수리,1364,정관신도시롯데캐슬,정관읍 용수리 1364 정관신도시롯데캐슬,101.7400,2008,201711,21~30,7,26700,35.336620,129.173120


In [71]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1216553 entries, 0 to 1216552
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   transaction_id          1216553 non-null  int64  
 1   apartment_id            1216553 non-null  int64  
 2   city                    1216553 non-null  object 
 3   dong                    1216553 non-null  object 
 4   jibun                   1216553 non-null  object 
 5   apt                     1216553 non-null  object 
 6   addr_kr                 1216553 non-null  object 
 7   exclusive_use_area      1216553 non-null  float64
 8   year_of_completion      1216553 non-null  int64  
 9   transaction_year_month  1216553 non-null  int64  
 10  transaction_date        1216553 non-null  object 
 11  floor                   1216553 non-null  int64  
 12  transaction_real_price  1216553 non-null  int64  
 13  latitude                1215455 non-null  float64
 14  lo

In [74]:
merged_df.to_csv(home + 'train_la&long.csv')

In [73]:
import zipfile

# 압축할 파일 및 압축 파일 이름 정의
file_to_compress = 'train_la&long.csv'
zip_file_name = 'train_la&long.zip'

# 압축할 파일의 경로
file_path = os.path.join(os.getcwd(), file_to_compress)

# 저장할 위치의 디렉토리
save_dir = os.path.join(os.getenv('HOME'), 'aiffel', 'datathon_apt', 'data')

# 압축 파일 경로 생성
zip_file_path = os.path.join(save_dir, zip_file_name)

# 압축 파일 생성
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(file_path, file_to_compress)

print(f'{zip_file_path} 파일이 생성되었습니다.')


/aiffel/aiffel/datathon_apt/data/train_la&long.zip 파일이 생성되었습니다.


In [24]:
unique_values.tail(40)

,city,dong,addr_kr,latitude,longitude
988598,부산광역시,신호동,부산광역시 신호동,35.087084,128.877051
988703,부산광역시,명지동,부산광역시 명지동,35.107136,128.917942
992210,부산광역시,지사동,부산광역시 지사동,35.147911,128.828341
996255,부산광역시,거제동,부산광역시 거제동,35.187620,129.069120
996337,부산광역시,연산동,부산광역시 연산동,35.183442,129.089934
1022493,부산광역시,망미동,부산광역시 망미동,35.175185,129.105014
1022523,부산광역시,수영동,부산광역시 수영동,35.170600,129.117050
1022529,부산광역시,민락동,부산광역시 민락동,35.158569,129.127121
1022554,부산광역시,광안동,부산광역시 광안동,35.159013,129.113064
1022677,부산광역시,남천동,부산광역시 남천동,35.144679,129.110088


In [20]:
data_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [19]:
unique_values.head()

,city,dong,addr_kr,latitude,longitude
0,서울특별시,신교동,서울특별시 신교동,37.584160,126.967230
1,서울특별시,필운동,서울특별시 필운동,37.577363,126.969097
3,서울특별시,내수동,서울특별시 내수동,37.572940,126.971800
5,서울특별시,인의동,서울특별시 인의동,37.572360,126.997100
10,서울특별시,이화동,서울특별시 이화동,37.580476,127.002547


In [22]:
merged_df = pd.merge(data_df, unique_values[['city', 'dong', 'latitude', 'longitude']], on=['city', 'dong'], how='left')

merged_df


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,latitude,longitude
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.8200,2002,200801,21~31,2,37500,37.584160,126.967230
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.1700,1973,200801,1~10,6,20000,37.577363,126.969097
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.7400,2007,200801,1~10,6,38500,37.577363,126.969097
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.3900,2003,200801,11~20,15,118000,37.572940,126.971800
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.4300,2004,200801,21~31,3,120000,37.572940,126.971800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216548,1234823,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,59.8245,2012,201711,11~20,3,22000,NaN,NaN
1216549,1234824,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,7,30750,NaN,NaN
1216550,1234825,9578,부산광역시,정관읍 용수리,1387,정관 동일스위트1차,정관읍 용수리 1387 정관 동일스위트1차,84.9923,2012,201711,21~30,9,29500,NaN,NaN
1216551,1234826,9586,부산광역시,정관읍 용수리,1364,정관신도시롯데캐슬,정관읍 용수리 1364 정관신도시롯데캐슬,101.7400,2008,201711,21~30,7,26700,NaN,NaN
